In [ ]:
import torch
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim


import torchvision
import torchvision.transforms as transforms
from torchsummary import summary

from tqdm import tqdm
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

torch.manual_seed(42)
if device == 'cuda':
    torch.cuda.manual_seed(42)

# 定义数据预处理和增强
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010)),
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

Using device: cuda


100%|██████████| 170M/170M [00:05<00:00, 31.0MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
model = nn.Sequential(
    # input_size(3, 32, 32)
    nn.Conv2d(3, 64, kernel_size=3, padding=1),
    nn.BatchNorm2d(64),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(64, 128, kernel_size=3, padding=1),
    nn.BatchNorm2d(128),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Conv2d(128, 256, kernel_size=3, padding=1),
    nn.BatchNorm2d(256),
    nn.ReLU(),
    nn.MaxPool2d(2, 2),

    nn.Flatten(),

    nn.Linear(256 * 4 * 4, 1024),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(1024, 10)
).to(device)

summary(model, input_size=(3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 32, 32]           1,792
       BatchNorm2d-2           [-1, 64, 32, 32]             128
              ReLU-3           [-1, 64, 32, 32]               0
         MaxPool2d-4           [-1, 64, 16, 16]               0
            Conv2d-5          [-1, 128, 16, 16]          73,856
       BatchNorm2d-6          [-1, 128, 16, 16]             256
              ReLU-7          [-1, 128, 16, 16]               0
         MaxPool2d-8            [-1, 128, 8, 8]               0
            Conv2d-9            [-1, 256, 8, 8]         295,168
      BatchNorm2d-10            [-1, 256, 8, 8]             512
             ReLU-11            [-1, 256, 8, 8]               0
        MaxPool2d-12            [-1, 256, 4, 4]               0
          Flatten-13                 [-1, 4096]               0
           Linear-14                 [-

In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Initialize the ReduceLROnPlateau scheduler without the deprecated 'verbose' parameter
LR_scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode="max",
    factor=0.1,
    patience=3,
    threshold=0.003,
    threshold_mode='rel',
    min_lr=1e-6
)

num_epochs = 50

# Early Stopping parameters
early_stopping_patience = 7  # Number of epochs to wait before stopping
counter = 0  # Counts the number of epochs without improvement
best_test_acc = 0.0  # Tracks the best test accuracy
early_stop = False  # Flag to trigger early stopping

# Model checkpoint path
best_model_path = "/content/drive/MyDrive/Model_Weights/CIFAR10/best_model.pth"

# Initialize previous learning rate
prev_lr = optimizer.param_groups[0]['lr']

for epoch in range(num_epochs):
    if early_stop:
        print("Early stopping triggered. Stopping training.")
        break  # Exit the training loop

    model.train()
    total_loss = 0.0
    total_samples = 0
    total_correct = 0

    with tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
        for i, (input, label) in enumerate(tepoch):
            input, label = input.to(device), label.to(device)
            optimizer.zero_grad()

            output = model(input)
            # loss: average loss of the batch
            loss = criterion(output, label)
            total_loss += loss.item()
            # len(train_loader): number of batches
            train_loss = total_loss / (i + 1)

            _, predicted = torch.max(output, 1)
            total_correct += (predicted == label).sum().item()
            # input.size(0): batch size; total_samples: number of inputs
            total_samples += input.size(0)
            train_acc = total_correct / total_samples

            loss.backward()
            optimizer.step()

            tepoch.set_postfix(train_loss=train_loss, train_accuracy=f"{100*train_acc:.2f}%")

    # Evaluation Phase
    model.eval()
    total_loss = 0.0
    total_samples = 0
    total_correct = 0

    with torch.no_grad():
        with tqdm(test_loader, desc=f"Evaluating Epoch {epoch+1}/{num_epochs}", unit="batch") as tepoch:
            for j, (input, label) in enumerate(tepoch):
                input, label = input.to(device), label.to(device)
                output = model(input)

                loss = criterion(output, label)
                total_loss += loss.item()
                test_loss = total_loss / (j + 1)  # Using j as counter

                _, predicted = torch.max(output, 1)
                total_correct += (predicted == label).sum().item()
                total_samples += input.size(0)
                test_acc = total_correct / total_samples

                tepoch.set_postfix(loss=test_loss, test_accuracy=f"{100*test_acc:.2f}%")

    # Step the scheduler with the test accuracy
    LR_scheduler.step(test_acc)

    # Get the current learning rate
    current_lr = optimizer.param_groups[0]['lr']

    # Print epoch metrics
    print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc*100:.2f}%")
    print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc*100:.2f}%")

    # Check if the learning rate has changed
    if current_lr != prev_lr:
        print(f"Learning Rate adjusted to: {current_lr}")
        prev_lr = current_lr

    # Check for improvement
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        counter = 0  # Reset counter since we have an improvement
        torch.save(model.state_dict(), best_model_path)  # Save the best model state
        print(f"New best test accuracy: {best_test_acc*100:.2f}% - Model saved.")
    else:
        counter += 1
        print(f"No improvement in test accuracy for {counter} epoch(s).")
        if counter >= early_stopping_patience:
            print(f"Early stopping triggered after {epoch+1} epochs.")
            early_stop = True  # Set the flag to stop training

    print("-" * 50)  # Separator for readability

# After training, load the best model state
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path))
    print(f"Loaded the best model from {best_model_path} with test accuracy: {best_test_acc*100:.2f}%")
else:
    print("No improvement observed during training.")

Evaluating Epoch 1/50: 100%|██████████| 313/313 [00:04<00:00, 73.98batch/s, loss=1.12, test_accuracy=59.46%]


Train Loss: 1.4614 | Train Accuracy: 47.31%
Test Loss: 1.1225 | Test Accuracy: 59.46%
New best test accuracy: 59.46% - Model saved.
--------------------------------------------------


Evaluating Epoch 2/50: 100%|██████████| 313/313 [00:05<00:00, 59.11batch/s, loss=0.894, test_accuracy=68.82%]


Train Loss: 1.0755 | Train Accuracy: 62.29%
Test Loss: 0.8937 | Test Accuracy: 68.82%
New best test accuracy: 68.82% - Model saved.
--------------------------------------------------


Evaluating Epoch 3/50: 100%|██████████| 313/313 [00:05<00:00, 58.00batch/s, loss=0.793, test_accuracy=72.56%]


...

Train Loss: 0.1399 | Train Accuracy: 95.08%
Test Loss: 0.5329 | Test Accuracy: 85.84%
No improvement in test accuracy for 5 epoch(s).
--------------------------------------------------


Evaluating Epoch 36/50: 100%|██████████| 313/313 [00:05<00:00, 55.74batch/s, loss=0.523, test_accuracy=85.82%]


Train Loss: 0.1379 | Train Accuracy: 95.19%
Test Loss: 0.5227 | Test Accuracy: 85.82%
No improvement in test accuracy for 6 epoch(s).
--------------------------------------------------


Evaluating Epoch 37/50: 100%|██████████| 313/313 [00:05<00:00, 59.25batch/s, loss=0.529, test_accuracy=85.85%]
<ipython-input-5-a8d88f61519e>:118: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this expe

Train Loss: 0.1331 | Train Accuracy: 95.46%
Test Loss: 0.5290 | Test Accuracy: 85.85%
No improvement in test accuracy for 7 epoch(s).
Early stopping triggered after 37 epochs.
--------------------------------------------------
Early stopping triggered. Stopping training.
Loaded the best model from /content/drive/MyDrive/Model_Weights/CIFAR10/best_model.pth with test accuracy: 85.94%
